## Name Extraction 
Extract person's names from article title using Stanford NE\
save the extracted person's name into txt files on disk for later use

## Face - Name Matching Dataset Creation
### I create an subset (face-name matching dataset) from the given image dataset. This dataset is only used for face-name matching
\
Create the image folder for each extracted name, and copy paired images to the name folder\
To reduce processing time, I apply several face detection algorithms on all images in both training dataset and test dataset. An image is removed from face-name matching dataset if no faces are detected.

In [77]:
def concat_name(classified_text):
    i=0
    name_list=[]
    while i < len(classified_text)-1:
        if classified_text[i][1] == 'PERSON':
            name = classified_text[i][0]
            if classified_text[i+1][1]=='PERSON':
                name+=" "+classified_text[i+1][0]
                i+=1 
            name_list.append(name)
        i+=1
    if i == len(classified_text)-1 and classified_text[i][1] == 'PERSON':
        name_list.append(classified_text[i][0])
    return name_list

In [ ]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os
java_path = r"C:\Users\yuxia\Documents\java-se-8u41-ri\bin\java.exe"
os.environ['JAVAHOME'] = java_path
st = StanfordNERTagger(r'C:\Users\yuxia\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\classifiers\english.all.3class.distsim.crf.ser.gz',
                       r'C:\Users\yuxia\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\stanford-ner.jar',
                       encoding='utf-8')
a_file = open(r"C:\Users\yuxia\Downloads\trans-test.txt")
next(a_file)
cnt=0
name_list=[]
for line in a_file:
    tokenized_text = word_tokenize(line)
    classified_text = st.tag(tokenized_text)
    names=concat_name(classified_text)
    if len(names) > 0:
        print(names)
        cnt+=1
    name_list.append(names)
print(cnt)

In [113]:
def comb_title_eng (orig_file, title_eng_file, output_file):
    lines = [line.strip() for line in open(orig_file, 'r', encoding="utf-8")]
    titles_eng = [line.strip() for line in open(title_eng_file, 'r', encoding="utf-8")]
    with open(output_file, 'a',encoding="utf-8") as the_file:
        for i in range(len(lines)):
            title_eng=titles_eng[i].rstrip("\n")
            segs=lines[i].strip("\n").split(",")
            the_file.write(segs[0]+"\t"+segs[1]+"\t"+segs[2]+'\t'+title_eng+"\n")

In [117]:
comb_title_eng("train_01.csv", "trans-01.txt", 'title_01_title_eng.tsv')

In [118]:
comb_title_eng("train_02.csv", "trans-02.txt",'title_02_title_eng.tsv')

In [119]:
comb_title_eng("train_03.csv", "trans-03.txt",'title_03_title_eng.tsv')

In [148]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os
java_path = r"C:\Users\yuxia\Documents\java-se-8u41-ri\bin\java.exe"
os.environ['JAVAHOME'] = java_path
st = StanfordNERTagger(r'C:\Users\yuxia\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\classifiers\english.all.3class.distsim.crf.ser.gz',
                       r'C:\Users\yuxia\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\stanford-ner.jar',
                       encoding='utf-8')

In [150]:
def add_title_name(tr_file, output_file):
    a_file = open(tr_file, encoding="utf8")
    next(a_file)
    cnt=0
    name_list=[]
    header="img_id"+"\t"+"img_name"+"\t"+"title"+"\t"+"title_eng"+"\t"+"title_names"
    with open(output_file, 'a',encoding="utf-8") as the_file:
        for line in a_file:
            title_eng=line.split("\t")[3]
            tokenized_text = word_tokenize(title_eng)
            classified_text = st.tag(tokenized_text)
            names=concat_name(classified_text)
            if len(names)>0:
                names_str = ','.join(names)
                print(names_str)
                new_line=line.strip("\n")+"\t"+names_str+"\n"
                cnt+=1
            else:
                new_line=line.strip("\n")+"\t "+"\n"
            the_file.write(new_line)
    print(cnt)

In [ ]:
add_title_name("title_02_title_eng.tsv", "title_02_title_eng_name.tsv")

In [ ]:
add_title_name("title_03_title_eng.tsv", "title_03_title_eng_name.tsv")

In [155]:
dataframe=pd.read_csv("title_03_title_eng_name - Copy.tsv", delimiter="\t")

In [156]:
dataframe

,129437.jpg,dpa-polizist-vor-polizeiauto.jpg,Polizei-Azubi erschießt mutmaßlich Mit-Auszubildenden,Police Azubi shoots alleged with-apprentices,
0,129439.jpg,ali-t--280219.jpg,Hauptverdächtiger in Türkei gefasst – und wied...,Main suspect in Turkey - and released again,
1,129440.jpg,kita-symbol.jpg,Rhein-Erft-Kreis stoppt Kita-Bonus für städtis...,Rhine-Erft-Kreis Stops Kita bonus for urban em...,Kita
2,129442.jpg,bahnsteig-station-dom-hauptbahnhof.jpg,KVB-Fahrgastzahlen sind Grund zur Freude und S...,KVB passenger numbers are reason for joy and c...,
3,129443.jpg,annegret-kramp-karrenbaur-cdu.jpg,„Wir wollen eine Rüstungsindustrie in Deutschl...,We want an armor industry in Germany,
4,129445.jpg,magier-jan-rouven.jpg,Deutscher Magier Jan Rouven in USA zu 20 Jahre...,German magician Jan Rouven sentenced to 20 yea...,Jan Rouven
...,...,...,...,...,...
2381,133721.jpg,bild-lei-containeruferstrasse-280319.jpg,Neue Gruppen in Containern,New groups in containers,
2382,133722.jpg,bild-bahnhofstrasse-leichlingen.jpg,Fahrbahnsanierung der Bahnhofstraße beginnt,Road renovation of the Bahnhofstraße begins,
2383,133726.jpg,urn-newsml-dpa-com-20090101-190328-99-579319-l...,Schwebebahn startet nach Unfall wieder Testfah...,Swabbahn starts test drives after accident,
2384,133727.jpg,5f9cf8003ab339b0.jpg,Kein Ende der Krise bei Bayer in Sicht,No end of the crisis at Bayer in sight,


In [201]:
import os
def comb_title_eng_text (orig_file, title_eng_file, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)
    texts=open(title_eng_file, 'r', encoding="utf-8")
    next(texts)
    lines = [line.strip() for line in open(orig_file, 'r', encoding="utf-8")]
    texts_eng = [line.strip() for line in texts]
    header="img_id"+"\t"+"img_name"+"\t"+"title"+"\t"+"title_eng"+"\t"+"title_names"+"\t"+"text_eng"+"\n"
    with open(output_file, 'a',encoding="utf-8") as the_file:
        the_file.write(header)
        for i in range(len(lines)):
            text_eng=texts_eng[i].rstrip("\n")
            orig_line=lines[i]
            if len(orig_line.split("\t"))!=5:
                orig_line+="\t"+"NA"
            the_file.write(orig_line+"\t"+text_eng+"\n")

In [202]:
comb_title_eng_text("title_01_title_eng_name.tsv", "trans_text_01_tr.txt", 'title_01_title_eng_name_text_1.tsv')

In [203]:
comb_title_eng_text("title_02_title_eng_name.tsv", "trans_text_02_tr.txt", 'title_02_title_eng_name_text.tsv')

In [204]:
comb_title_eng_text("title_03_title_eng_name.tsv", "trans_text_03_tr.txt", 'title_03_title_eng_name_text.tsv')

In [ ]:
comb_title_eng_text("title_03_title_eng_name.tsv", "trans_text_03_tr.txt", 'title_03_title_eng_name_text.tsv')

In [192]:
def add_text_name(tr_file, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)
    a_file = open(tr_file, encoding="utf8")
    next(a_file)
    cnt=0
    name_list=[]
    header="img_id"+"\t"+"img_name"+"\t"+"title"+"\t"+"title_eng"+"\t"+"title_names"+"\t"+"text_eng"+"\t"+"text_names"+"\n"
    with open(output_file, 'a',encoding="utf-8") as the_file:
        the_file.write(header)
        for line in a_file:
            title_eng=line.split("\t")[5]
            tokenized_text = word_tokenize(title_eng)
            classified_text = st.tag(tokenized_text)
            names=concat_name(classified_text)
            if len(names)>0:
                names_str = ','.join(names)
                print(names_str)
                new_line=line.strip("\n")+"\t"+names_str+"\n"
                cnt+=1
            else:
                new_line=line.strip("\n")+"\t "+"\n"
            the_file.write(new_line)
    print(cnt)

In [ ]:
add_text_name('title_01_title_eng_name_text_1.tsv', 'final_01.tsv')

In [213]:
from shutil import copyfile
def create_name_folder (name_file):
    a_file = open(name_file, encoding="utf8")
    next(a_file)
    for line in a_file:
        line=line.strip("\n")
        img_name=line.split("\t")[0]
        names=line.split("\t")[4].rstrip()
        if len(names)>0:
            path=os.path.join("faces", names.split(",")[0])
            if not os.path.exists(path):
                os.makedirs(path)
            copyfile(os.path.join("train_img",img_name) , os.path.join(path,img_name))

In [217]:
create_name_folder("title_01_title_eng_name.tsv")

In [218]:
create_name_folder("title_02_title_eng_name.tsv")

In [219]:
create_name_folder("title_03_title_eng_name.tsv")

In [222]:
df=pd.read_csv(r"C:\Users\yuxia\Documents\CS7311_Project\FIREWHEEL\data\MediaEvalNewsImagesBatch04images.tsv", delimiter="\t")
df

,img,iid,imgFile
0,https://www.ksta.de/image/23526828/2x1/300/150...,134801.0,23526828a8202ba73ce31c14cd42b066452c2f8SR.jpg
1,https://www.ksta.de/image/23575686/2x1/300/150...,136751.0,23575686742e1a5e86aa6f2f919ea19f3fe10d54Rh.jpg
2,https://www.ksta.de/image/23575776/2x1/300/150...,135527.0,2357577628a56ed4968b8c369921ebb39b271d4dEs.jpg
3,https://www.ksta.de/image/23575790/2x1/300/150...,135908.0,23575790cb42cc1a43b7d8996d4c5b6f586185b8aj.jpg
4,https://www.ksta.de/image/23575818/2x1/300/150...,135583.0,23575818c61b4d0961b6e0bb2798b78fbd901885qb.jpg
...,...,...,...
1911,https://www.ksta.de/image/32435912/2x1/300/150...,136928.0,324359122cc0aff2a1eb75cff09f948d63fcdf6ezg.jpg
1912,https://www.ksta.de/image/32436164/2x1/300/150...,136931.0,32436164c26ad3a09ab4604cde6a60df95ae0ca3AG.jpg
1913,https://www.ksta.de/image/32436674/2x1/300/150...,136958.0,32436674531b23b9bf5b73d1ada08a89c218bf82zA.jpg
1914,https://www.ksta.de/image/32437042/2x1/300/150...,136941.0,32437042a64866cc07ab616bc74dd6a741b7eb19Vn.jpg


In [ ]:
import os
import requests  # to get image from the web
import shutil

f = open(os.path.join(r'C:\Users\yuxia\Documents\CS7311_Project\FIREWHEEL\data',"MediaEvalNewsImagesBatch04images.tsv"), "r",encoding="utf-8")
next(f)
for line in f:
    image_url = line.split("\t")[0]
    image_id=line.split("\t")[1]
    filename = os.path.join('test_img',image_id+".jpg")
    r = requests.get(image_url, stream=True,headers={'User-agent': 'Mozilla/5.0'})
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
    else:
        print("img can't be loaded")

In [224]:
df=pd.read_csv(r"C:\Users\yuxia\Documents\CS7311_Project\FIREWHEEL\data\MediaEvalNewsImagesBatch04articles.tsv", delimiter="\t")
df

,articleID,aid,url,Unnamed: 3,Unnamed: 4,hashvalue,title,text,nImpressions,nRecs,nClicks
0,1.000265e+09,NaN,https://www.ksta.de/panorama/mit-kopfschuss-tr...,NaN,NaN,NaN,Trainspotting-Star auf offener Straße erschossen,In Schottland haben Unbekannte den Schauspiele...,13.0,0.0,0.0
1,1.001935e+09,NaN,https://www.ksta.de/koeln/verspaetungen-und-au...,NaN,NaN,NaN,S-Bahn-Gleis am Kölner Hauptbahnhof ist gesperrt,Der S-Bahn-Bahnsteig am Kölner Hauptbahnhof is...,35.0,0.0,0.0
2,1.002375e+09,NaN,https://www.ksta.de/koeln/muelheim/zwei-kilo-m...,NaN,NaN,NaN,Drogendealer in Mülheim auf frischer Tat ertappt,Die Polizei hat einen einen 22-jährigen Drogen...,437.0,25.0,0.0
3,1.002735e+09,NaN,https://www.ksta.de/koeln/fruehlingsvolksfest-...,NaN,NaN,NaN,Die ungewöhnliche Ostermesse auf dem Autoscooter,"Mit fast 33 Metern Höhe, einer Streckenlänge v...",178.0,0.0,0.0
4,1.002835e+09,NaN,https://www.ksta.de/region/rhein-erft/bergheim...,NaN,NaN,NaN,Polizei fasst flüchtende Einbrecher nach 40 Se...,Das war ein kurzer Ausflug für zwei Einbrecher...,394.0,543.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
1911,1.999165e+09,NaN,https://www.ksta.de/sport/fussball/auch-hambur...,NaN,NaN,NaN,RB Leipzig gewinnt dramatischen Pokalfight in ...,RB Leipzig darf dank Marcel Halstenberg weiter...,8.0,4.0,0.0
1912,1.999345e+09,NaN,https://www.ksta.de/panorama/berliner-eisbaerc...,NaN,NaN,NaN,Berliner Eisbärchen heißt Hertha,Begeisterung für Bälle hat sie schon gezeigt: ...,6.0,642.0,2.0
1913,1.999355e+09,NaN,https://www.ksta.de/politik/rnd/nach-moorbrand...,NaN,NaN,NaN,Bund zahlt 600.000 Euro Schadensersatz,Für Schäden nach einem Moorbrand auf dem Gelän...,5.0,679.0,0.0
1914,1.999735e+09,NaN,https://www.ksta.de/region/rwe-stoppt-boaplus-...,NaN,NaN,NaN,„Kohlekraftwerke haben keinen Platz in Zukunfs...,Der Energiekonzern RWE hat die Planungen zum B...,12.0,0.0,0.0


In [233]:
import os
def extract_columns (orig_file, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)
    articles=open(orig_file, 'r', encoding="utf-8")
    next(articles)
    lines = [line.strip() for line in articles]
    header="article_id"+"\t"+"title"+"\n"
    with open(output_file, 'a',encoding="utf-8") as the_file:
        the_file.write(header)
        for i in range(len(lines)):
            orig_line=lines[i]
            segs = orig_line.split("\t")
            print(len(segs))
            print(segs)
            the_file.write(segs[0]+"\t"+segs[6]+"\n")

In [ ]:
extract_columns(r"C:\Users\yuxia\Documents\CS7311_Project\FIREWHEEL\data\MediaEvalNewsImagesBatch04articles.tsv", "test_orig.tsv")

In [236]:
def comb_title_eng_test (orig_file, title_eng_file, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)
    lines = [line.strip() for line in open(orig_file, 'r', encoding="utf-8")]
    titles_eng = [line.strip() for line in open(title_eng_file, 'r', encoding="utf-8")]
    with open(output_file, 'a',encoding="utf-8") as the_file:
        for i in range(len(lines)):
            title_eng=titles_eng[i].rstrip("\n")
            the_file.write(lines[i]+"\t"+title_eng+"\n")

In [237]:
comb_title_eng_test("test_orig.tsv", "trans-test.txt", "test_title_eng.tsv")

In [239]:
def add_title_name_text(tr_file, output_file):
    a_file = open(tr_file, encoding="utf8")
    next(a_file)
    cnt=0
    name_list=[]
    header="artitle_id"+"\t"+"title_de"+"\t"+"title_eng"+"\t"+"title_names"
    with open(output_file, 'a',encoding="utf-8") as the_file:
        for line in a_file:
            title_eng=line.split("\t")[2]
            tokenized_text = word_tokenize(title_eng)
            classified_text = st.tag(tokenized_text)
            names=concat_name(classified_text)
            if len(names)>0:
                names_str = ','.join(names)
                print(names_str)
                new_line=line.strip("\n")+"\t"+names_str+"\n"
                cnt+=1
            else:
                new_line=line.strip("\n")+"\t "+"NA"+"\n"
            the_file.write(new_line)
    print(cnt)

In [ ]:
add_title_name_text("test_title_eng.tsv","test_title_eng_name.tsv")

In [2]:
import os
def extract_text (orig_file):
    articles_names=open(orig_file, 'r', encoding="utf-8")
    next(articles_names)
    lines = [line.strip() for line in articles_names]
    result=[]
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        result.append((segs[0], segs[3].split(",")[0]))
    return result

In [3]:
ar_name_list=extract_text("test_title_eng_name.tsv")

In [4]:
len(ar_name_list)

1914

In [ ]:
d='faces'
sub_directories = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
idx_name={}
name_idx={}
idx=1
for sub_dir in sub_directories:
    idx_name[sub_dir]='face_'+str(idx)
    name_idx['face_'+str(idx)] = sub_dir
    idx+=1
name_idx

In [29]:
import os
import shutil
def copy_all_files(src, dest, idx_name):
    src_files = os.listdir(src)
    for file_name in src_files:
        full_file_name = os.path.join(src, file_name)
        dest_folder=os.path.join(dest, idx_name[os.path.basename(src)])
        if not os.path.isdir(dest_folder):
            os.mkdir(dest_folder)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dest_folder)

In [ ]:
d='faces'
sub_full_paths = [os.path.join(d, o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
sub_full_paths

In [31]:
mapped_folder='mapped_face'
for sub_dir in sub_full_paths:
    copy_all_files(sub_dir, mapped_folder, idx_name)

In [ ]:
from deepface import DeepFace
cnt = 0
img_files = [f for f in os.listdir("test_img") if f.endswith('.jpg')]
ar_img_files={}
for ar_name in ar_name_list:
    if ar_name[1].strip() != 'NA' and ar_name[1] in idx_name:
        if os.path.exists(os.path.join("mapped_face", idx_name[ar_name[1]])):
            df_results=[]
            for img_file in img_files:
                img_path=os.path.join("test_img", img_file)
                df = DeepFace.find(img_path = img_path, db_path =os.path.join("mapped_face", idx_name[ar_name[1]]), enforce_detection=False)
                df_results.append(df)
            ar_img_files[ar_name[0]]=df_results
            print(len(df_results))
            cnt+=1
print(cnt)

In [279]:
print(cnt)

92


In [280]:
len(ar_img_files)

92

In [286]:
next(iter(ar_img_files))

'1003715259'

In [ ]:
from deepface import DeepFace
img_files = [f for f in os.listdir("test_img") if f.endswith('.jpg')]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
c=0
for img_f in img_files:
    try:
        backends = ['ssd', 'mtcnn']
        detected_face = DeepFace.detectFace(os.path.join('test_img',img_f), detector_backend = backends[0])
        shutil.copy(os.path.join('test_img',img_f), "test_img_face")
        c+=1
    except Exception:
        print("no face")
        
print(c)

In [8]:
df_1 = DeepFace.find(img_path = r"test_img_face\134332.jpg", db_path = r"faces\Altmaier", model_name="VGG-Face")

There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.4572765827178955  seconds


In [9]:
df_1

,identity,VGG-Face_cosine
0,faces\Altmaier/124941.jpg,0.398045


In [13]:
from deepface import DeepFace
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
def deep_detect(file):
    c=0
    for model in models:
        try:
            detected_face = DeepFace.detectFace(file, model_name=model)
        except:
            c+=1
    if c== len(models):
        return False
    else:
        return True

In [33]:
import cv2
import time
import matplotlib.pyplot as plt
def detect_face_cv(file):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # Read the input image
    img = cv2.imread(file)
    width, height, channels=img.shape
    # Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces= face_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw rectangle around the faces
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        if w!=width or height!=h:
            return True
    return False

In [ ]:
path="mapped_face"
sub_directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
c=0
for sub_dir in sub_directories:
    files = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith('.jpg')]
    for file in files:
        if not detect_face_cv(file) and not deep_detect(file):
            shutil.copy(file, "mapped_no_face")

In [36]:
import os
import shutil
path="mapped_face"
path_no_face="mapped_no_face"
sub_directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
files_no_face = [f for f in os.listdir(path_no_face) if f.endswith('.jpg')]
c=0
for sub_dir in sub_directories:
    files = [f for f in os.listdir(sub_dir) if f.endswith('.jpg')]
    for f in files:
        if f in files_no_face:
            shutil.move(os.path.join(sub_dir,f), "mapped_face_removed")
            

In [37]:
for sub_dir in sub_directories:
    files = [f for f in os.listdir(sub_dir)]
    if len(files)==0:
        os.rmdir(sub_dir)

In [ ]:
d='faces'
sub_directories = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
idx_name={}
name_idx={}
idx=1
for sub_dir in sub_directories:
    idx_name[sub_dir]='face_'+str(idx)
    name_idx['face_'+str(idx)] = sub_dir
    idx+=1
name_idx

In [ ]:
from deepface import DeepFace
img_files = [os.path.join('test_img',f) for f in os.listdir("test_img") if f.endswith('.jpg')]
c=0
for img_f in img_files:
    if  detect_face_cv(img_f) or deep_detect(img_f):
        shutil.copy(img_f, "test_img_face")
        c+=1
    else:
        print("no face")
print(c)

In [25]:
from deepface import DeepFace
def deep_detect_backend(file):
    backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']
    c=0
    for backend in backends:
        try:
            detected_face = DeepFace.detectFace(file, detector_backend=backend)
        except:
            c+=1
    if c== len(backends):
        return False
    else:
        return True

In [ ]:
from deepface import DeepFace
img_files = [os.path.join('test_img',f) for f in os.listdir("test_img") if f.endswith('.jpg')]
c=0
for img_f in img_files:
    if  deep_detect_backend(img_f): 
        shutil.copy(img_f, "test_img_face")
        c+=1
    else:
        print("no face")
print(c)

In [34]:
path="mapped_face"
sub_directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
c=0
for sub_dir in sub_directories:
    files = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith('.jpg')]
    for file in files:
        if not detect_face_cv(file) and not deep_detect(file) and not deep_detect_backend(file) :
            shutil.copy(file, "mapped_no_face")

In [38]:
df_1 = DeepFace.find(img_path = r"test_img_face\134332.jpg", db_path = r"faces\Altmaier", model_name=models[0])

There are  1  representations found in  representations_vgg_face.pkl
find function lasts  1.3980741500854492  seconds


In [39]:
df_1

,identity,VGG-Face_cosine
0,faces\Altmaier/124941.jpg,0.398045


In [ ]:
import os
from deepface import DeepFace
cnt = 0
img_files = [f for f in os.listdir("test_img_face") if f.endswith('.jpg')]
ar_img_files={}
for ar_name in ar_name_list:
    if ar_name[1].strip() != 'NA' and ar_name[1] in idx_name:
        if os.path.exists(os.path.join("mapped_face", idx_name[ar_name[1]])):
            df_results=[]
            for img_file in img_files:
                img_path=os.path.join("test_img_face", img_file)
                df = DeepFace.find(img_path = img_path, db_path =os.path.join("mapped_face", idx_name[ar_name[1]]), model_name='Facenet', enforce_detection=False)
                df_results.append((img_path,df))
            ar_img_files[ar_name[0]]=df_results
            cnt+=1
            print(cnt, " name completed")

In [51]:
print(len(ar_img_files))

128
